In [1]:
from matplotlib import pyplot as plt
import numpy as np
import json
import math
import csv
import os

In [2]:
def denormalization(arr, angle, translation):

    theta = (angle)/180*math.pi
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))
    
    #rotate
    arr = np.array(R.dot(arr.transpose())).transpose()
    #translate
    arr += translation
#     arr[...,0] += (AGENT[19,0] - AGENT[0,0])
#     arr[...,1] += (AGENT[19,1] - AGENT[0,1])
    return arr

In [20]:
def prediction(json_dict):
    preds = np.array(json_dict['preds'][0])[...,:2]
    preds = np.array([denormalization(p, -json_dict['rotation'], -np.array(json_dict['translation'])) for p in preds])
    seq_id = int(json_dict['csv_file'].split('.')[0])
    return preds, seq_id

In [4]:
from tqdm.notebook import tqdm
from argoverse.evaluation.competition_util import generate_forecasting_h5


In [10]:
file_name = "origianl_test.json"
output_path = ["competition_files/" + file_name]



In [6]:
with open(file_name) as json_data:
    data = json.load(json_data)

In [12]:
print(file_name)
print(output_path)


origianl_test.json
['competition_files/origianl_test.json']


In [14]:
data[0].keys()

dict_keys(['preds', 'waypoint_preds', 'rotation', 'translation', 'csv_file', 'city', 'agent_labels', 'agent_features', 'social_features', 'social_label_features', 'att_weights'])

In [19]:
for k in data[0].keys():
    try:
        print(k , " :", np.array(data[0][k]).shape)
    except:
        pass

preds  : (1, 6, 30, 3)
waypoint_preds  : (2,)
rotation  : ()
translation  : (2,)
csv_file  : ()
city  : ()
agent_labels  : (30, 2)
agent_features  : (20, 2)
social_features  : (24, 20, 2)
social_label_features  : (24, 30, 2)
att_weights  : (4, 25, 25)


<ipython-input-19-ea3f984524d1>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  print(k , " :", np.array(data[0][k]).shape)


In [40]:
output_all = {}
for d in tqdm(data):
    p, seq_id = prediction(d)
    output_all[seq_id] = p
generate_forecasting_h5(output_all, output_path[0])

  0%|          | 0/78143 [00:00<?, ?it/s]

9192/78143

limit_output extension: Maximum message size of 100000 exceeded with 103646 characters

In [39]:
output_path =  "competition_files/"